Installing Packages

In [1]:
!pip install -U langchain langchain-community openai langfuse yfinance python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.5/720.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.4/275.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 939.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.81.0
    Uninstalling openai-1.81.0:
      Successfully uninstalled openai-1.81.0


Import Libraries

In [2]:
import os, warnings, logging
warnings.filterwarnings("ignore")
logging.getLogger("langchain").setLevel(logging.ERROR)

from langchain_community.chat_models import AzureChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
import yfinance as yf
from dotenv import load_dotenv
from langfuse import Langfuse


Load Environment Variables

In [3]:
from google.colab import files
uploaded = files.upload()


Saving Secrets.env to Secrets.env


In [5]:
from dotenv import load_dotenv
load_dotenv("Secrets.env")  # Make sure the filename is exactly `.env`


True

In [6]:
import os

print(os.getenv("OPENAI_ENDPOINT"))
print(os.getenv("LANGFUSE_PUBLIC_KEY")[:6], "...")  # Just to check without revealing full


https://genaiarchassignment1.openai.azure.com/
pk-lf- ...


Define Output Schema Using Pydantic

In [7]:
class SentimentOutput(BaseModel):
    company_name: str
    stock_code: str
    newsdesc: str
    sentiment: str
    people_names: list[str] = Field(default_factory=list)
    places_names: list[str] = Field(default_factory=list)
    other_companies_referred: list[str] = Field(default_factory=list)
    related_industries: list[str] = Field(default_factory=list)
    market_implications: str
    confidence_score: float


Define Prompt and Azure GPT LLM

In [8]:
parser = PydanticOutputParser(pydantic_object=SentimentOutput)

prompt = PromptTemplate(
    template="""
You are a market analyst AI. Analyze this company news:

{newsdesc}

Company: {company_name}, Stock Code: {stock_code}

{format_instructions}
""",
    input_variables=["newsdesc", "company_name", "stock_code"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

llm = AzureChatOpenAI(
    openai_api_base=os.getenv("OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    deployment_name=os.getenv("DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("OPENAI_API_VERSION")
)


<ipython-input-8-a84ddc5c968a>:17: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


News Fetching Function

In [9]:
def get_news_summary(company):
    ticker_obj = yf.Ticker(company)
    ticker = ticker_obj.info.get("symbol", company)
    news_items = ticker_obj.news[:3]
    newsdesc = " ".join([item["title"] for item in news_items])
    return newsdesc, ticker


Langfuse Integration + Sentiment Analysis

In [10]:
langfuse = Langfuse(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
    host=os.getenv("LANGFUSE_HOST")
)

In [16]:
def analyze_sentiment(newsdesc, company, ticker):
    trace = langfuse.trace(name="SentimentChain")

    span1 = trace.span(name="Prompt Construction")
    prompt_input = prompt.format_prompt(
        newsdesc=newsdesc,
        company_name=company,
        stock_code=ticker
    )
    span1.end()

    span2 = trace.span(name="LLM Prediction")
    response = llm.invoke(prompt_input.to_string())  # ✅ new method
    span2.end()

    span3 = trace.span(name="Output Parsing")
    result = parser.parse(response.content)
    span3.end()

    # ❌ Do NOT call trace.end() — not needed with Langfuse client
    return result


Running the Pipeline

In [17]:
company = "Microsoft"
newsdesc, ticker = get_news_summary(company)
result = analyze_sentiment(newsdesc, company, ticker)

# Show structured result
print(result.model_dump_json(indent=2))



{
  "company_name": "Microsoft",
  "stock_code": "MICROSOFT",
  "newsdesc": "Microsoft has announced a new partnership with OpenAI to enhance its AI capabilities across various products.",
  "sentiment": "positive",
  "people_names": [
    "Satya Nadella",
    "Sam Altman"
  ],
  "places_names": [
    "Redmond",
    "San Francisco"
  ],
  "other_companies_referred": [
    "OpenAI"
  ],
  "related_industries": [
    "Technology",
    "Artificial Intelligence"
  ],
  "market_implications": "This partnership is expected to strengthen Microsoft's position in the AI market and could lead to increased revenue streams from AI-integrated products.",
  "confidence_score": 0.85
}
